# Mettalex Autonomous Market Maker
Date: 2021-01-07

Author: Matt McDonnell, Fetch.ai/Mettalex

This notebook describes calcuations used to design an improved version of the Mettalex Autonomous Market Maker (AMM) to provide fair liquidity-sensitive pricing.  We present a new approach using [method of characteristics](https://en.wikipedia.org/wiki/Method_of_characteristics) solutions of partial differential equations (PDEs) to examine existing AMM invariants such as Uniswap, Balancer and Curve (a.k.a. Stableswap).  This approach is then extended to take into account the additional constraints imposed by the Mettalex position token price invariants.

Currently the Mettalex system is running on the Ethereum blockchain which has the benefit of allowing reuse of existing DeFi protocols at the expense of some limitations in the allowed efficient mathematical operations. The Mettalex system roadmap plans to transition the autonomous market making functionality to the Fetch.ai network to make use of more advanced optimization techniques. As such the functionality described here should be regarded as an intial version of the AMM that will be extended as more network capabilities become available.

The initial version of the Mettalex AMM described in [amm_balancer.ipynb](amm_balancer.ipynb) makes use of a private Balancer Smart Pool internally to provide the functionality of swapping between coin token (USDT intially) and long or short position tokens. The internal Balancer AMM is controlled to set prices of long and short tokens to reflect market demand and constraints on the token prices.

The new version of the Mettalex AMM discussed in this notebook builds on the analysis of the previous notebook to propose changes to the Balancer invariant to achieve system constraints.

In [1]:
# Initial notebook setup: functions are stored in ./calc/amm_math.py 
%load_ext autoreload
%autoreload 2

from IPython.display import HTML
# Hide code cells https://gist.github.com/uolter/970adfedf44962b47d32347d262fe9be  
def hide_code():
    return HTML('''<script>
    code_show=true; 
    function code_toggle() {
        if (code_show){
        $("div.input").hide();
        } else {
        $("div.input").show();
        }
    code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    The raw code for this IPython notebook is by default hidden for easier reading.
    To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'calc'))

from amm_math import (
    calc_spot_price, calc_token_balance, calc_out_given_in, calc_in_given_out, 
    set_amm_state, set_amm_state_rebalance, get_amm_spot_prices, get_amm_balance, 
    simple_swap_from_coin, simple_swap_to_coin, simulate_swaps_from_coin,
    calc_balancer_invariant, mint_redeem, deposit_withdraw, perform_action,
    plot_action, plot_action_orderbook, print_state_change, perform_action_sequence
)

import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [2]:
hide_code()

In [3]:
# Weights
w_c, w_l, w_s = sp.symbols('w_c w_l w_s', positive=True)
# Token balances
x_c, x_l, x_s = sp.symbols('x_c x_l x_s', positive=True)
# Prices - in units of locked up collateral
v = sp.symbols('v', positive=True) 
# Swap fee in fraction of input tokens
s_f = sp.symbols('s_f')
# Decimal scaling 
d_c, d_p = sp.symbols('d_c d_p', positive=True)
# Collateral backing L + S 
# e.g. for $100.000_000_000_000_000_000 = 1.000_000 mt
# this is 100*10**18 / 1*10**6 = 10**14
C = sp.symbols('C', positive=True)

# Input token amounts for swaps
a_c, a_l, a_s = sp.symbols('a_c a_l a_s', positive=True)
x_m = sp.symbols('x_m')  # Amount of coin tokens used for minting long and short

From the [Balancer whitepaper](https://balancer.finance/whitepaper/):

> The bedrock of Balancer’s exchange functions is a surface defined by constraining a value function $V$
 — a function of the pool’s weights and balances — to a constant. We will prove that this surface implies a spot price at each point such that, no matter what exchanges are carried out, the share of value of each token in the pool remains constant.

In [4]:
V_0 = calc_balancer_invariant(x_c, x_l, x_s, w_c, w_l, w_s)

In [5]:
V_0

x_c**w_c*x_l**w_l*x_s**w_s

The [Balancer whitepaper](https://balancer.finance/whitepaper/) shows that the value function is related to the token spot prices by the ratio of partial derivative.  This is shown below by comparing the partial derivative ratio with the explict expression for the spot price.

In [6]:
# Spot price is given by tangent to invariant
sp.Eq(sp.Derivative(V_0,x_l)/sp.Derivative(V_0, x_c), V_0.diff(x_l)/V_0.diff(x_c))

Eq(Derivative(x_c**w_c*x_l**w_l*x_s**w_s, x_l)/Derivative(x_c**w_c*x_l**w_l*x_s**w_s, x_c), w_l*x_c/(w_c*x_l))

# Starting from the Constraints
The idea of constant level sets of a value function creating constraints on system state (including prices) is discussed in ['From Curved Bonding to Configuration Spaces'](https://epub.wu.ac.at/7385/) by Zargham, Shorish, and Paruch (2020).  

The existing Balancer value function implicit state constraint is 'the share of value of each token in the pool remains constant'.  We seek to introduce a new constraint 'the sum of position token spot prices equals the underlying collateral' for Mettalex position tokens.

In this section we look at starting from a set of constraints and see if it is possible to derive the corresponding value function.  We can then use this value function to determine allowed state changes e.g. for a swap the number of output tokens for an initial state and given number of input tokens.

This approach feels familiar to the Lagrangian dynamics approach in classical physics (the author's background).  In economics the standard approach seems to be start from a value function (a.k.a. utility function) and derive substitution functions that give prices.  Here we attempt to solve the inverse problem.

As a starting point, we consider deriving the Balancer value function (a [Cobb-Douglas Utility Function](https://www.wikiwand.com/en/Cobb%E2%80%93Douglas_production_function)) from the set of constraints for swaps 'the share of value of each token in the pool remains constant'.  

The state of the system can be defined by three token balances $x_c$, $x_l$, $x_s$, and three weights $w_c$, $w_l$, $w_s$, giving 6 variables in total.

The share of value constraints on token swaps gives 3 constraints, and we can choose to introduce a further token weight sum to 1 constraint.  This leaves 2 free parameters and by inspection we see that the value function we hope to derive is invariant to scaling of the function (and possibly all parameters).  While this proves nothing it is at least reassuring that we might be on the right track.

## Uniswap
The two asset case is a generalized form of [Uniswap](https://uniswap.org/docs/v2/protocol-overview/how-uniswap-works/) having token weights $w_x$ and $w_y$ summing to 1.  Uniswap uses $w_x = w_y = \frac{1}{2}$.  Here we use $x$ and $y$ to represent the token X and token Y balances.

The total value of the pool in tems of token X is  
$$x - \frac{\partial{x}}{\partial{y}}y$$
i.e. number of tokens X plus the spot price of converting Y tokens into X tokens.

The constant share of value constraint is hence represented by the equations: 
$$w_x = \frac{x}{x - \frac{\partial{x}}{\partial{y}}y} \\
w_y = \frac{- \frac{\partial{x}}{\partial{y}}y}{x - \frac{\partial{x}}{\partial{y}}y}$$
where in the two asset case the second equation is redundant since the weights sum to 1. 

In [7]:
x, y, w_x, w_y = sp.symbols('x y w_x w_y', positive=True)
k = sp.symbols('k', real=True)
X, Y = map(sp.Function, 'XY')
V = sp.Function('V')

We can specify that swaps happen on some invariant surface $V(x,y)$ which allows us to replace the spot price $-\frac{\partial{x}}{\partial{y}}$ in  $x - y\frac{\partial{x}}{\partial{y}} = w_x$, subsitituting $-\frac{\partial{x}}{\partial{y}} = \frac{\partial{V}}{\partial{y}}/\frac{\partial{V}}{\partial{x}}$ via the [implicit function theorem](https://en.wikipedia.org/wiki/Implicit_function_theorem).

In [8]:
sp.Eq(x/(x + y*V(x,y).diff(y)/V(x,y).diff(x)), w_x)

Eq(x/(x + y*Derivative(V(x, y), y)/Derivative(V(x, y), x)), w_x)

SymPy's PDE solver balks at this equation as written, so multiply through to make things easier.

In [9]:
const_share_eq = sp.Eq(x*V(x,y).diff(x), w_x*(x*V(x,y).diff(x) + y*V(x,y).diff(y)))

In [10]:
const_share_eq

Eq(x*Derivative(V(x, y), x), w_x*(x*Derivative(V(x, y), x) + y*Derivative(V(x, y), y)))

It turns out that SymPy is capable of solving the PDE directly to give a general solution.

In [11]:
V_sol = sp.pdsolve(const_share_eq).subs({(1-w_x): w_y})

In [12]:
V_sol

Eq(V(x, y), F(-log(x**(-w_x)*y**(-w_y))/w_y))

We can simplify the solution:

In [13]:
V_sol.rhs.simplify()

F(w_x*log(x)/w_y + log(y))

We show below that the spot price is as expected regardless of the exact form of $F$ and if a specific form is chosen we achieve the desired Cobb-Douglas form.  

In [14]:
sp.Eq((V(x,y).diff(y)/V(x,y).diff(x)), V_sol.rhs.diff(y)/V_sol.rhs.diff(x))

Eq(Derivative(V(x, y), y)/Derivative(V(x, y), x), w_y*x/(w_x*y))

By taking the exponential of the constant we obtain the general Uniswap invariant.

In [15]:
sp.exp(w_y*V_sol.rhs.args[0]).simplify()

x**w_x*y**w_y

Interestingly, the general solution offers the opportunity to use different functional forms to achieve the same constant share of value contraint.  

In [16]:
(w_y*V_sol.rhs.args[0]).expand()

w_x*log(x) + w_y*log(y)

We could derive the swap formulae for each form.  Would the formulae from log form of invariant be easier to implement on the Ethererum blockchain?  This is left as an exercise for the reader.

## Balancer
We now consider the general case of a Balancer pool with $n$ assets and weights summing to 1.  Here we examine the three asset case and identify the geometric constraints imposed by the share of value conditions
$$
w_x = \frac{x}{x - \frac{\partial{x}}{\partial{y}}y - \frac{\partial{x}}{\partial{z}}z}\\
w_y = \frac{-\frac{\partial{x}}{\partial{y}}y}{x - \frac{\partial{x}}{\partial{y}}y - \frac{\partial{x}}{\partial{z}}z}\\
w_z = \frac{-\frac{\partial{x}}{\partial{z}}z}{x - \frac{\partial{x}}{\partial{y}}y - \frac{\partial{x}}{\partial{z}}z}
$$
for tokens X, Y and Z having total value $x - \frac{\partial{x}}{\partial{y}}y - \frac{\partial{x}}{\partial{z}}z$.

As before we can replace the spot prices given by the partial derivatives with expressions for the invariant surface V.  Below we show the $w_x$ condition, the others are similar.  The weight sum to 1 condition again allows us to eliminate one of the constraint equations.

In [17]:
x, y, z, w_x, w_y, w_z = sp.symbols('x y z w_x w_y w_z', positive=True)
u, v, w = sp.symbols('u v w', positive=True)
xi = sp.Function('xi')
eta = sp.Function('eta')
zeta = sp.Function('zeta')
V = sp.Function('V')

In [18]:
const_share_eq_3_1 = sp.Eq(x*V(x,y,z).diff(x), w_x*(x*V(x,y,z).diff(x) + y*V(x,y,z).diff(y) + z*V(x,y,z).diff(z)))

In [19]:
const_share_eq_3_1

Eq(x*Derivative(V(x, y, z), x), w_x*(x*Derivative(V(x, y, z), x) + y*Derivative(V(x, y, z), y) + z*Derivative(V(x, y, z), z)))

We can simplify this to a constant coefficient first order PDE by the change of variables 
$$
u = \xi(x) = \log{x}\\
v = \eta(y) = \log{y}\\
w = \zeta(z) = \log{z}
$$

In [20]:
const_share_eq_3_1.subs({
    V(x,y,z): V(xi(x),eta(y),zeta(z)),
}).simplify()

Eq(x*Derivative(V(xi(x), eta(y), zeta(z)), xi(x))*Derivative(xi(x), x), w_x*(x*Derivative(V(xi(x), eta(y), zeta(z)), xi(x))*Derivative(xi(x), x) + y*Derivative(V(xi(x), eta(y), zeta(z)), eta(y))*Derivative(eta(y), y) + z*Derivative(V(xi(x), eta(y), zeta(z)), zeta(z))*Derivative(zeta(z), z)))

In [21]:
const_share_eq_3_1.subs({
    V(x,y,z): V(xi(x),eta(y),zeta(z)),
    xi(x): sp.log(x),
    eta(y): sp.log(y), 
    zeta(z): sp.log(z)
}).simplify()

Eq(Subs(Derivative(V(_xi_1, log(y), log(z)), _xi_1), _xi_1, log(x)), w_x*(Subs(Derivative(V(_xi_1, log(y), log(z)), _xi_1), _xi_1, log(x)) + Subs(Derivative(V(log(x), _xi_2, log(z)), _xi_2), _xi_2, log(y)) + Subs(Derivative(V(log(x), log(y), _xi_3), _xi_3), _xi_3, log(z))))

The share of value weight constraints can written as a matrix equation:

In [22]:
balancer_constraints = sp.ImmutableMatrix([
    [1-w_x, -w_x, -w_x],
    [-w_y, 1-w_y, -w_y],
    [-w_z, -w_z, 1-w_z]
])
v_grad = sp.ImmutableMatrix([V(u,v,w).diff(i) for i in [u,v,w]])

In [23]:
sp.Eq(sp.MatMul(balancer_constraints, sp.UnevaluatedExpr(v_grad), evaluate=False),
      balancer_constraints*v_grad, 
     evaluate=False)

Eq(Matrix([
[1 - w_x,    -w_x,    -w_x],
[   -w_y, 1 - w_y,    -w_y],
[   -w_z,    -w_z, 1 - w_z]])*Matrix([
[Derivative(V(u, v, w), u)],
[Derivative(V(u, v, w), v)],
[Derivative(V(u, v, w), w)]]), Matrix([
[-w_x*Derivative(V(u, v, w), v) - w_x*Derivative(V(u, v, w), w) + (1 - w_x)*Derivative(V(u, v, w), u)],
[-w_y*Derivative(V(u, v, w), u) - w_y*Derivative(V(u, v, w), w) + (1 - w_y)*Derivative(V(u, v, w), v)],
[-w_z*Derivative(V(u, v, w), u) - w_z*Derivative(V(u, v, w), v) + (1 - w_z)*Derivative(V(u, v, w), w)]]))

We can use the weights summing to 1 condition to eliminate one of these equations.  The nullspace of the constraint matrix then defines a plane of constant value of the invariant in $u,v,w$ space which we can then map back to the original X, Y, Z token balances.

In [24]:
(sp.simplify(sp.ImmutableMatrix([
#     [1-w_x, -w_x, -w_x],
    [-w_y, 1-w_y, -w_y],
    [-w_z, -w_z, 1-w_z]
]).nullspace()[0]).subs({(1-w_y-w_z): w_x})*w_z
).T*sp.ImmutableMatrix([sp.log(x), sp.log(y), sp.log(z)])

Matrix([[w_x*log(x) + w_y*log(y) + w_z*log(z)]])

It can be seen by inspection that exponentiating this invariant results in the original Balancer form.  As for the two asset case it is also possible to retain the value function in this form and derive new forms for the trading functions, which is again an exercise left to the reader.

## Curve 
Curve uses the [StableSwap](https://www.curve.fi/stableswap-paper.pdf) invariant to reduce slippage for stablecoins all having equivalent value.  For example a pool could consist of USDC, USDT, BUSD and DAI, all of which are designed to track USD.  Pools tracking other assets are possible e.g. a BTC pool backed by sBTC, renBTC, wBTC.  The market maker token pool ideally consists of a balanced mix of each token type.  We consider a two asset pool below, this analysis extends to an arbitrary number of assets.

The StableSwap invariant is designed to act as a constant sum market maker $x+y=1$ for small imbalances, and a constant product Uniswap market maker $xy=k$ as the pool becomes more imbalanced.  These are the price constraints defining the system i.e.

* at small imbalance $-\frac{\partial{x}}{\partial{y}}=1$ and tokens are freely interchangeable
* at larger imbalance $-\frac{\partial{x}}{\partial{y}}=x/y$ as for Uniswap (or in general an equal weight Balancer pool)

The StableSwap invariant can be written as $V{\left(x,y \right)} = s \left(x + y\right) + x^{w_{x}} y^{w_{y}}$ where $s$ is an amplification parameter that determines the transition between constant sum and constant product behaviour.


In [25]:
s = sp.symbols('s', positive=True)

In [26]:
V_ss = s*(x+y) + x**w_x*y**w_y

In [27]:
sp.Eq(V(x,y), V_ss)

Eq(V(x, y), s*(x + y) + x**w_x*y**w_y)

Below we show the spot price of Y tokens in terms of X tokens that results from this invariant function.  We can see that the limit $s \rightarrow \infty$ gives the constant sum behaviour while the $s \rightarrow 0$ limit gives constant product behaviour.

In [28]:
V_x = (V_ss).diff(x).subs({w_x: sp.Rational(1,2), w_y: sp.Rational(1,2)}).simplify()
V_y = (V_ss).diff(y).subs({w_x: sp.Rational(1,2), w_y: sp.Rational(1,2)}).simplify()
ss_spot = V_y/V_x.simplify()
ss_spot_eq = sp.Eq(V(x,y).diff(y)/V(x,y).diff(x), ss_spot)
ss_spot_eq
# sp.pdsolve(V_x*V(x,y).diff(y) - V_y*V(x,y).diff(x), V(x,y))
# sp.pdsolve(V_x*V(x,y).diff(y) - V_y*V(x,y).diff(x), V(x,y))
# (V_y/V_x).simplify()


Eq(Derivative(V(x, y), y)/Derivative(V(x, y), x), (s + sqrt(x)/(2*sqrt(y)))/(s + sqrt(y)/(2*sqrt(x))))

In [29]:
sp.Eq(sp.Limit(ss_spot, s, sp.oo), (ss_spot).limit(s, sp.oo))

Eq(Limit((s + sqrt(x)/(2*sqrt(y)))/(s + sqrt(y)/(2*sqrt(x))), s, oo, dir='-'), 1)

In [30]:
sp.Eq(sp.Limit(ss_spot, s, 0), (ss_spot).limit(s, 0))

Eq(Limit((s + sqrt(x)/(2*sqrt(y)))/(s + sqrt(y)/(2*sqrt(x))), s, 0), x/y)

Following the previous procedure we'd hope to be able to solve the PDE for $V(x,y)$ from the spot price constraint:
$$\left(s + \frac{\sqrt{y}}{2 \sqrt{x}}\right) \frac{\partial}{\partial y} V{\left(x,y \right)} = \left(s + \frac{\sqrt{x}}{2 \sqrt{y}}\right) \frac{\partial}{\partial x} V{\left(x,y \right)}$$
Attempting this with the actual StableSwap spot price above doesn't immediately work although it should be possible to solve numerically.  

In [31]:
ss_spot_denom = sp.denom(ss_spot_eq.rhs)* sp.denom(ss_spot_eq.lhs)
sp.Eq(ss_spot_eq.lhs * ss_spot_denom, ss_spot_eq.rhs * ss_spot_denom)

Eq((s + sqrt(y)/(2*sqrt(x)))*Derivative(V(x, y), y), (s + sqrt(x)/(2*sqrt(y)))*Derivative(V(x, y), x))

## A new Curve
We can look at the form of the PDE from the StableSwap constraint and explore related functional forms.  It looks like the same limiting behaviour could be achieved with any ratio of $x/y$ i.e. without requiring the $\surd$.  We hence try $$\left(s x y + y\right) \frac{\partial}{\partial y} V{\left(x,y \right)} = \left(s x y + x\right) \frac{\partial}{\partial x} V{\left(x,y \right)}$$
which is easily solvable by SymPy and results in a new Curve invariant:

In [32]:
sol2 = sp.pdsolve((s*x*y + y)*V(x,y).diff(y) - (s*x*y + x)*V(x,y).diff(x), V(x,y)).rhs
V_ss_new = sp.log(sol2.args[0]).expand()
sp.Eq(V(x,y),V_ss_new)

Eq(V(x, y), s*x + s*y + log(x) + log(y))

We see that the spot price has the same desired limiting behaviour: 

In [33]:
ss_spot_new = (V_ss_new.diff(y)/V_ss_new.diff(x)).simplify()

In [34]:
ss_spot_new

x*(s*y + 1)/(y*(s*x + 1))

In [35]:
sp.Eq(sp.Limit(ss_spot_new, s, sp.oo), (ss_spot_new).limit(s, sp.oo))

Eq(Limit(x*(s*y + 1)/(y*(s*x + 1)), s, oo, dir='-'), 1)

In [36]:
sp.Eq(sp.Limit(ss_spot_new, s, 0), (ss_spot_new).limit(s, 0))

Eq(Limit(x*(s*y + 1)/(y*(s*x + 1)), s, 0), x/y)

Comparing to the characteristic surfaces obtained for Uniswap and Balancer using this approach, we see that the functional form is quite similar i.e. a plane in $\log{x}-\log{y}$ space now with the addition of an exponential surface for the imbalance part.  

A nice feature of this new form is the possibility of investigating weighted Curve pools, and this may be useful for introducing the necessary constraints for Mettalex position tokens.

# Mettalex
The Mettalex AMM consists of a pool containing coin (collateral), long and short tokens.  To simplify notation we will use
$$x = x_c\\y = x_l\\z = x_s
$$
for the token balances and $w_x, w_y, w_z$ for the corresponding weights.  

The position tokens track an underlying asset price that is supplied to the AMM by an oracle.  A pair of long and short position tokens is backed by collateral $C$ proportional to the price range of the allowed floor to cap trading band.  Without loss of generality we set $C = p_{cap} - p_{floor}$ and assume a multiplier of 1 here.

Within the allowed trading band the fair value of the tokens are 
$$v_{long} = v C\\v_{short}=(1-v) C$$
where 
$$v = \frac{p_{spot}-p_{floor}}{p_{cap}-p_{floor}}$$

We start by modelling the Mettalex AMM as a Balancer pool with dynamic weights, and use spot price $p_y$ in place of $-\frac{\partial{x}}{\partial{y}}$ etc to simplify notation.  The share of weight for each token is then given by
$$
w_x = \frac{x}{x + p_y y + p_z z}\\
w_y = \frac{p_y y}{x + p_y y + p_z z}\\
w_z = \frac{p_z z}{x + p_y y + p_z z}
$$
We can eliminate the $w_x$ equation via the weight sum constraint, leaving a system of 2 equations and 2 unknowns for the prices.

In [37]:
p_y, p_z = sp.symbols('p_y p_z', positive=True)

In [38]:
pool_total = x + y*p_y + z*p_z
price_sum = p_y + p_z
prices = sp.solve([p_y*y/pool_total - w_y, p_z*z/pool_total - w_z], [p_y, p_z])

The expression of the sum of prices reduces to a simple form, where $w_x = 1 - w_y - w_z$ has been substituted in below.

In [39]:
sp.Eq(price_sum.subs(prices).subs({1-w_y-w_z: w_x}).simplify(), C)

Eq(x*(w_y*z + w_z*y)/(w_x*y*z), C)

We can introduce another equation for the token prices at zero imbalance i.e. at $y=z$ we have 
$$
\frac{p_y}{p_z} = \frac{v}{1-v}
$$

In [40]:
sp.Eq((p_y/p_z).subs(prices), v/(1-v)).subs({z: y})

Eq(w_y/w_z, v/(1 - v))

In [41]:
w_z_inter = sp.solve(sp.Eq((p_y/p_z).subs(prices), v/(1-v)).subs({z: y}), w_z)[0]

We can then solve for $w_y$ and hence for all weights

In [43]:
w_y_sol = sp.solve(
    sp.Eq(price_sum.subs(prices).subs({w_z: w_z_inter}).simplify(), C), w_y
)[0].collect([v,x]).simplify()

In [44]:
w_z_sol = w_z_inter.subs({w_y: w_y_sol}).simplify()

In [46]:
w_x_sol = (1- w_y_sol - w_z_sol).simplify().collect([v,x])

## Balance Sensitive Weights
Below we show these Balancer pool weights needed to take into account position token imbalance for a given underlying scaled asset price $v$.

In [47]:
sp.Eq(w_x, w_x_sol)

Eq(w_x, x*(v*(y - z) - y)/(-C*y*z + v*(x*y - x*z) - x*y))

In [48]:
sp.Eq(w_y, w_y_sol)

Eq(w_y, C*v*y*z/(C*y*z - v*x*(y - z) + x*y))

In [49]:
sp.Eq(w_z, w_z_sol)

Eq(w_z, C*y*z*(1 - v)/(C*y*z - v*x*(y - z) + x*y))

### Spot price check
For a Balancer pool the spot price of token Y in terms of token X is given by 
$$
p_y = \frac{\frac{x}{w_x}}{\frac{y}{w_y}} = \frac{w_y}{w_x}\frac{x}{y}
$$
as shown in Spot Price section, equation 2 of the [Balancer whitepaper](https://balancer.finance/whitepaper/).

We check below that the spot prices for long (Y) and short (Z) tokens at zero imbalance $y = z$ are as expected.

In [50]:
spot_y_limit = sp.Limit((w_y_sol/w_x_sol*x/y).simplify(), z, y)
sp.Eq(spot_y_limit, spot_y_limit.simplify())

Eq(Limit(-C*v*z/(v*(y - z) - y), z, y), C*v)

In [51]:
spot_z_limit = sp.Limit((w_z_sol/w_x_sol*x/z).simplify(), z, y)
sp.Eq(spot_z_limit, spot_z_limit.simplify().collect(C))

Eq(Limit(C*y*(v - 1)/(v*(y - z) - y), z, y), C*(1 - v))

### Comparison with previous version
A previous version of the Mettalex AMM used a simpler approach to setting the Balancer pool weights.  The constraints

* $w_x + w_y + w_z = 1$ for weights
* $\frac{w_y}{w_x}\frac{x}{y} = v C$ for long token price
* $\frac{w_z}{w_x}\frac{x}{z} = (1-v) C$ for short token price

can be used to solve for weights that meet an arbitrary price target.  The simple approach taken was to calculate rescaled position token prices after a swap so that the sum of the prices equaled $C$, then calculate the corresponding weights and update the Balancer pool parameters.

The original simple weights are shown below.

In [52]:
_, _, _, w_x_simple, w_y_simple, w_z_simple = set_amm_state(x,y,z,v,C)

In [53]:
sp.Eq(w_x, w_x_simple)

Eq(w_x, x/(C*v*y - C*z*(v - 1) + x))

In [54]:
sp.Eq(w_y, w_y_simple)

Eq(w_y, C*v*y/(C*v*y - C*z*(v - 1) + x))

In [55]:
sp.Eq(w_z, w_z_simple)

Eq(w_z, -C*z*(v - 1)/(C*v*y - C*z*(v - 1) + x))

The drawback of the simple approach is the ad hoc approach to rescaling token prices introduces large changes to the Balancer pool invariant.  It is left as an exercise to the reader to compare the relative weight changes for a given amount swapped between the simple weight calculation and the Balance Sensitive Weights.

# Future work
The above **balance sensitive weights** are sufficient to create a Balancer smart pool that takes into account the imbalance between long and short positions.  It can also respond to underlying asset price updates by reweighting.  
Some straight forward algebraic areas of future investigation are:

* Weight impact calculation in terms of a given amount of swapped tokens
* Comparison between the impact on balance sensitive weights and simple weights as mentioned in previous section
* Changing the weights of the pool also changes the Balancer invariant.  We can however mint or redeem pairs of long and short position tokens to maintain the invariant.  Can we solve for the amount to mint or redeem analytically?

A more fundamental task would be to use the PDE approach discussed in the bulk of this paper to derive a new invariant function for the Mettalex AMM.  This should just involve solving the expression for $p_y$ in terms of invariant function
$$
p_y = -\frac{\partial{x}}{\partial{y}} = \frac{\frac{\partial{V}}{\partial{y}}}{\frac{\partial{V}}{\partial{x}}} = \frac{vCyz}{x(v(y-z)-y)}
$$

(work in progress below)

In [56]:
prices[p_y].subs({w_y: w_y_sol, w_z: w_z_sol}).simplify().collect(v)

C*v*z/(v*(-y + z) + y)

In [57]:
(w_z_sol/w_x_sol).subs({y: z}).simplify()*x/z

-C*(v - 1)

In [58]:
sp.series((w_y_sol/w_x_sol).subs({z: y + d_p}).simplify()*x/y, d_p, n=3)

d_p*(-C*v**2/y + C*v/y) + d_p**2*(C*v**3/y**2 - C*v**2/y**2) + C*v + O(d_p**3)

In [59]:
sp.series((w_y_sol/w_x_sol).subs({y: z - d_p}).simplify()*x/z, d_p, n=3)

d_p**2*(C*v**3/z**2 - C*v**2/z**2) + C*v - C*d_p*v**2/z + O(d_p**3)

In [60]:
set_amm_state(x,y,z, v, C)

[x,
 y,
 z,
 x/(C*v*y - C*z*(v - 1) + x),
 C*v*y/(C*v*y - C*z*(v - 1) + x),
 -C*z*(v - 1)/(C*v*y - C*z*(v - 1) + x)]